# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import osmnx as ox
import networkx
import random
import time
from tqdm import tqdm


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Methods

In [5]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [6]:
def gen_linkers(road_type_factors, link_factor):
    linkables = [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
    ]

    for linkable in linkables:
        if linkable in road_type_factors:
            road_type_factors[linkable + "_link"] = road_type_factors[linkable] * link_factor
    
    return road_type_factors


In [7]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times):
    total_diff = 0
    total_diff_percentage = 0

    for path, acceptable_range in benchmark_times.items():
        if path in calculated_times:

            calculated_time = calculated_times[path]
            lower_bound, upper_bound = acceptable_range

            if lower_bound <= calculated_time <= upper_bound:
                diff = 0
                diff_percentage = 0
            else:
                diff = min(abs(calculated_time - lower_bound), abs(calculated_time - upper_bound))
                diff_percentage = min((diff / lower_bound) * 100, (diff / upper_bound) * 100)

            total_diff += diff
            total_diff_percentage += diff_percentage

            print(f"Path {path}:")
            print(f"  Benchmark Time: {acceptable_range} minutes")
            print(f"  Calculated Time: {calculated_time} minutes")
            print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
            print()
        else:
            print(f"Path {path} not found in calculated times.")

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all paths.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")


In [8]:
def calculate_error(od: pathing.OriginDestination, benchmark_times):
    calculated_times = {}
    
    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # store the calculated travel time
        calculated_times[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time

    # calculate MSE
    errors = []

    for path, acceptable_range in benchmark_times.items():
        calculated_time = calculated_times[path]
        lower_bound, upper_bound = acceptable_range

        # Check if the calculated time falls within the acceptable range
        if lower_bound <= calculated_time <= upper_bound:
            error = 0
        else:
            # if outside the range, find the smallest absolute difference to the bounds
            error = min(abs(calculated_time - lower_bound), abs(calculated_time - upper_bound))
        
        errors.append(error)

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return mse, calculated_times


In [9]:
def optimize_parameters(od: pathing.OriginDestination, benchmark_times, max_duration_minutes):
    start_time = time.time()
    max_duration_seconds = max_duration_minutes * 60
    
    # define parameter bounds
    param_bounds = {
        "default_intersection_penalty": (1, 15),
        "traffic_signal_penalty": (1, 15),
        "linkable_factor": (0.1, 1.0),
        "road_type_factors": {
            "motorway": (0.1, 1.0),
            "trunk": (0.1, 1.0),
            "primary": (0.1, 1.0),
            "secondary": (0.1, 1.0),
            "tertiary": (0.1, 1.0),
            "unclassified": (0.1, 1.0),
            "residential": (0.1, 1.0),
            "living_street": (0.1, 1.0)
        }
    }
    
    # initialize the best solution
    best_solution = None
    best_error = float("inf")
    
    with tqdm(desc="Optimizing Parameters") as pbar:
        while time.time() - start_time < max_duration_seconds:
            # generate a random solution within the bounds
            current_solution = {
                "default_intersection_penalty": random.randint(*param_bounds["default_intersection_penalty"]),
                "traffic_signal_penalty": random.randint(*param_bounds["traffic_signal_penalty"]),
                "road_type_factors": {k: random.uniform(*v) for k, v in param_bounds["road_type_factors"].items()}
            }
            linkable_factor = random.uniform(*param_bounds["linkable_factor"])

            current_solution["road_type_factors"] = gen_linkers(current_solution["road_type_factors"], linkable_factor)
            
            # set graph weights with the current solution
            od.set_graph_weights_v2(**current_solution)
            
            # calculate the error for the current solution
            current_error, _ = calculate_error(od, benchmark_times)
            
            # update best solution if the current one is better
            if current_error < best_error:
                best_solution = current_solution
                best_linkable_factor = linkable_factor
                best_error = current_error
                tqdm.write(
                    f"    New best solution:\n"
                    f"        Default Intersection Penalty: {best_solution['default_intersection_penalty']}\n"
                    f"        Traffic Signal Penalty: {best_solution['traffic_signal_penalty']}\n"
                    f"        Linkable Factor: {best_linkable_factor}\n"
                    f"        Road Type Factors:\n"
                    + "\n".join([f"            {k}: {v}" for k, v in best_solution['road_type_factors'].items()]) +
                    f"\n        Error: {best_error}"
                )
            
            pbar.update()

            # early stop
            if current_error == 0:
                break
    
    return best_solution, best_error


In [10]:
def fix_lat_lon(benchmark_times):
    new_benchmark_times = {}

    for ((start_lat, start_lon), (end_lat, end_lon)), benchmark in benchmark_times.items():
        new_benchmark_times[((start_lon, start_lat), (end_lon, end_lat))] = benchmark

    return new_benchmark_times

# Main

In [11]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)


In [12]:
# times from Google Maps (Sunday, start at 02:00)
benchmark_times = {
    ((59.9369806, 10.7343661), (59.6800045, 10.6572104)): (35, 45),
    ((59.93596, 10.73244), (59.92629, 10.77570)): (4, 9),
    ((59.90864, 10.73921), (59.93037, 10.77273)): (9, 16),
    ((59.92727, 10.73174), (59.86305, 10.66617)): (50, 60),
    ((59.82052, 10.47168), (59.95577, 11.04773)): (40, 50),
    ((60.00352, 10.76216), (59.83905, 10.80742)): (28, 28),

    ((60.31262, 11.14674), (60.11964, 11.47577)): (40, 40),
    ((60.11964, 11.47577), (60.13423, 11.166858)): (22, 26),
    ((60.13423, 11.166858), (59.879044, 11.561687)): (50, 50),
    ((59.879044, 11.561687), (59.932076, 10.987775)): (40, 40),
    ((59.932076, 10.987775), (60.042152, 10.880784)): (20, 22),
    ((60.042152, 10.880784), (59.930733, 10.831094)): (20, 20),
    ((59.930733, 10.831094), (59.917, 10.758744)): (9, 14),
    ((59.917, 10.758744), (59.93917, 10.741926)): (7, 14),
    ((59.93917, 10.741926), (59.71566, 10.853331)): (35, 35),
    ((59.71566, 10.853331), (59.659687, 10.725881)): (18, 18),
    ((59.659687, 10.725881), (59.833073, 10.806946)): (22, 22),
    ((59.833073, 10.806946), (59.830055, 10.439887)): (30, 40),
    ((59.830055, 10.439887), (59.89813, 10.509723)): (14, 14),
    ((59.89813, 10.509723), (59.939663, 10.687367)): (16, 20),
    ((59.939663, 10.687367), (59.893173, 10.806364)): (12, 16),
    ((59.893173, 10.806364), (59.960304, 10.884091)): (12, 18),
    ((59.960304, 10.884091), (59.997875, 11.03928)): (20, 20),
    ((59.997875, 11.03928), (59.917873, 10.585751)): (28, 35),
    ((59.917873, 10.585751), (60.31262, 11.14674)): (55, 65),
}

benchmark_times = fix_lat_lon(benchmark_times)


In [14]:
best_solution, best_error = optimize_parameters(
    od,
    benchmark_times,
    max_duration_minutes=420
)


Optimizing Parameters: 1it [00:05,  5.27s/it]

    New best solution:
        Default Intersection Penalty: 8
        Traffic Signal Penalty: 3
        Linkable Factor: 0.7609600604154423
        Road Type Factors:
            motorway: 0.24631683612922928
            trunk: 0.4843809919684464
            primary: 0.6597034903183263
            secondary: 0.8708388224147655
            tertiary: 0.6205587230516473
            unclassified: 0.14743737090216003
            residential: 0.15330424640853896
            living_street: 0.19778772557242896
            motorway_link: 0.1874372745022389
            trunk_link: 0.3685945889124008
            primary_link: 0.5020080078489118
            secondary_link: 0.6626735629168525
            tertiary_link: 0.4722204033847112
        Error: 350.9347405865898


Optimizing Parameters: 3it [00:15,  5.05s/it]

    New best solution:
        Default Intersection Penalty: 9
        Traffic Signal Penalty: 6
        Linkable Factor: 0.8280278271715367
        Road Type Factors:
            motorway: 0.4528499824881419
            trunk: 0.5110197817800957
            primary: 0.650874753778817
            secondary: 0.6026768551940588
            tertiary: 0.7525099544563536
            unclassified: 0.14267547562046673
            residential: 0.584050036077447
            living_street: 0.6167267239814744
            motorway_link: 0.37497238703432456
            trunk_link: 0.4231385995490454
            primary_link: 0.5389424081322828
            secondary_link: 0.4990332068929113
            tertiary_link: 0.6230991825134465
        Error: 323.6201456995785


Optimizing Parameters: 8it [00:40,  5.00s/it]

...

Optimizing Parameters: 102it [09:22,  5.35s/it]

    New best solution:
        Default Intersection Penalty: 4
        Traffic Signal Penalty: 3
        Linkable Factor: 0.5645044249485629
        Road Type Factors:
            motorway: 0.9396322884756202
            trunk: 0.8621624455331938
            primary: 0.923209418713176
            secondary: 0.965321293921126
            tertiary: 0.44277979792247424
            unclassified: 0.691761205090562
            residential: 0.8040707660782066
            living_street: 0.7589327341981997
            motorway_link: 0.5304265846690321
            trunk_link: 0.4866945155279622
            primary_link: 0.5211558020177784
            secondary_link: 0.5449281419155478
            tertiary_link: 0.2499511552050672
        Error: 4.414926475898013


Optimizing Parameters: 137it [12:42,  5.73s/it]

    New best solution:
        Default Intersection Penalty: 2
        Traffic Signal Penalty: 5
        Linkable Factor: 0.2630284973443693
        Road Type Factors:
            motorway: 0.9704026402720763
            trunk: 0.9872728848094453
            primary: 0.9434615680765118
            secondary: 0.49062545330175955
            tertiary: 0.9327873614166583
            unclassified: 0.8502498894729571
            residential: 0.7543590170147566
            living_street: 0.8078248863334735
            motorway_link: 0.25524354828977275
            trunk_link: 0.259680903360269
            primary_link: 0.24815727855332728
            secondary_link: 0.12904847574086184
            tertiary_link: 0.24534965801524275
        Error: 1.385063884849518


Optimizing Parameters: 4908it [7:00:00,  5.13s/it]


In [15]:
od.set_graph_weights_v2(**best_solution)

_, calculated_times = calculate_error(od, benchmark_times)

evaluate_travel_time_accuracy(benchmark_times, calculated_times)


Path ((10.7343661, 59.9369806), (10.6572104, 59.6800045)):
  Benchmark Time: (35, 45) minutes
  Calculated Time: 37.89051763353399 minutes
  Difference: 0 minutes (0.00%)

Path ((10.73244, 59.93596), (10.7757, 59.92629)):
  Benchmark Time: (4, 9) minutes
  Calculated Time: 5.373738068070134 minutes
  Difference: 0 minutes (0.00%)

Path ((10.73921, 59.90864), (10.77273, 59.93037)):
  Benchmark Time: (9, 16) minutes
  Calculated Time: 8.732545058105803 minutes
  Difference: 0.26745494189419716 minutes (1.67%)

Path ((10.73174, 59.92727), (10.66617, 59.86305)):
  Benchmark Time: (50, 60) minutes
  Calculated Time: 52.471294431270884 minutes
  Difference: 0 minutes (0.00%)

Path ((10.47168, 59.82052), (11.04773, 59.95577)):
  Benchmark Time: (40, 50) minutes
  Calculated Time: 39.843186508386786 minutes
  Difference: 0.15681349161321378 minutes (0.31%)

Path ((10.76216, 60.00352), (10.80742, 59.83905)):
  Benchmark Time: (28, 28) minutes
  Calculated Time: 29.92681830724972 minutes
  Diffe